In [1]:
import os
import cv2
import numpy as np
import copy
from tqdm import tqdm
import requests
from PIL import Image 

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_tensor
from torchvision import transforms as TF
import torch.nn.functional as F
from torch.optim import AdamW

from transformers import SegformerForSemanticSegmentation
from transformers import get_scheduler

from sklearn.metrics import jaccard_score

class BDDDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None, num_images=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = [img for img in os.listdir(images_dir) if img.endswith('.jpg')]

        # Ensure the dataset has the same number of images and masks
        if num_images:
            if num_images > len(self.images):
                print(f"Requested {num_images} images, but only {len(self.images)} available. Using all available images.")
            else:
                self.images = self.images[:num_images]

        self.masks = [img.replace('.jpg', '.png') for img in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.masks[idx])
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert('L')  # Convert mask to grayscale
        
        # Convert mask to binary format with 0 and 1 values
        mask = np.array(mask)
        mask = (mask > 0).astype(np.uint8)  # Assuming non-zero pixels are lanes
        
        # Convert to PIL Image for consistency in transforms
        mask = Image.fromarray(mask)

        if self.transform:
            image = self.transform(image)
        
        mask = TF.functional.resize(img=mask, size=[360, 640], interpolation=Image.NEAREST)
        mask = TF.functional.to_tensor(mask)
        mask = (mask > 0).long()  # Threshold back to binary and convert to LongTensor

        return image, mask

def mean_iou(preds, labels, num_classes):
    # Flatten predictions and labels
    preds_flat = preds.view(-1)
    labels_flat = labels.view(-1)

    # Check that the number of elements in the flattened predictions
    # and labels are equal
    if preds_flat.shape[0] != labels_flat.shape[0]:
        raise ValueError(f"Predictions and labels have mismatched shapes: "
                         f"{preds_flat.shape} vs {labels_flat.shape}")

    # Calculate the Jaccard score for each class
    iou = jaccard_score(labels_flat.cpu().numpy(), preds_flat.cpu().numpy(),
                        average=None, labels=range(num_classes))

    # Return the mean IoU
    return np.mean(iou)

# Define the appropriate transformations
transform = TF.Compose([
    TF.Resize((360, 640)),
    TF.ToTensor(),
    TF.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the dataset with the specified number of images
train_dataset = BDDDataset(images_dir='/kaggle/input/lanedetection/deep_drive_10K/train/images',
                           masks_dir='/kaggle/input/lanedetection/deep_drive_10K/train/masks',
                           transform=transform,
                           num_images=1000)

valid_dataset = BDDDataset(images_dir='/kaggle/input/lanedetection/deep_drive_10K/valid/images',
                           masks_dir='/kaggle/input/lanedetection/deep_drive_10K/valid/masks',
                           transform=transform,
                           num_images=250)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=6)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=6)

# Load the pre-trained model
model = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b2-finetuned-ade-512-512')

# Adjust the number of classes for BDD dataset
model.config.num_labels = 2  # Replace with the actual number of classes

# Check for CUDA acceleration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the learning rate scheduler
num_epochs = 20
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Placeholder for best mean IoU and best model weights
best_iou = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

# Lists to store the loss values
train_losses = []
valid_ious = []

for epoch in range(num_epochs):
    model.train()
    train_iterator = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch")
    epoch_train_loss = 0.0

    for batch in train_iterator:
        images, masks = batch
        images = images.to(device)
        masks = masks.to(device).long()  # Ensure masks are LongTensors

        # Remove the channel dimension from the masks tensor
        masks = masks.squeeze(1)  # This changes the shape from [batch, 1, H, W] to [batch, H, W]
        optimizer.zero_grad()

        # Pass pixel_values and labels to the model
        outputs = model(pixel_values=images, labels=masks, return_dict=True)
        
        loss = outputs["loss"]
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        outputs = F.interpolate(outputs["logits"], size=masks.shape[-2:], mode="bilinear", align_corners=False)
        
        epoch_train_loss += loss.item()
        train_iterator.set_postfix(loss=loss.item())

    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Evaluation loop for each epoch
    model.eval()
    total_iou = 0
    num_batches = 0
    valid_iterator = tqdm(valid_loader, desc="Validation", unit="batch")

    for batch in valid_iterator:
        images, masks = batch
        images = images.to(device)
        masks = masks.to(device).long()
    
        with torch.no_grad():
            # Get the logits from the model and apply argmax to get the predictions
            outputs = model(pixel_values=images, return_dict=True)
            outputs = F.interpolate(outputs["logits"], size=masks.shape[-2:], mode="bilinear", align_corners=False)
            preds = torch.argmax(outputs, dim=1)
            preds = torch.unsqueeze(preds, dim=1)

        preds = preds.view(-1)
        masks = masks.view(-1)
    
        # Compute IoU
        iou = mean_iou(preds, masks, model.config.num_labels)
        total_iou += iou
        num_batches += 1
        valid_iterator.set_postfix(mean_iou=iou)
    
    epoch_iou = total_iou / num_batches
    valid_ious.append(epoch_iou)
    print(f"Epoch {epoch+1}/{num_epochs} - Mean IoU: {epoch_iou:.4f}")

    # Check for improvement
    if epoch_iou > best_iou:
        print(f"Validation IoU improved from {best_iou:.4f} to {epoch_iou:.4f}")
        best_iou = epoch_iou
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, '/kaggle/working/best_model.pth')

# After all epochs, load the best model weights - optional


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


config.json:   0%|          | 0.00/6.88k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/110M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Epoch 1/20:   0%|          | 0/250 [00:00<?, ?batch/s]

...

Epoch 2/20:  86%|████████▌ | 214/250 [02:49<00:27,  1.29batch/s, loss=0.0737]

Epoch 2/20:  86%|████████▌ | 215/250 [02:49<00:27,  1.28batch/s, loss=0.0737]

Epoch 2/20:  86%|████████▌ | 215/250 [02:50<00:27,  1.28batch/s, loss=0.0755]

Epoch 2/20:  86%|████████▋ | 216/250 [02:50<00:26,  1.28batch/s, loss=0.0755]

Epoch 2/20:  86%|████████▋ | 216/250 [02:51<00:26,  1.28batch/s, loss=0.0831]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(valid_ious, label='Validation IoU')
plt.xlabel('Epoch')
plt.ylabel('Mean IoU')
plt.title('Validation Mean IoU')
plt.legend()

plt.show()

In [ ]:
# Code written by Pranav Durai for Lane Detection Inference using Fine-Tuned SegFormer Model
# Import necessary libraries
import cv2
from PIL import Image
import torch
import numpy as np
from torchvision import transforms as TF
import torch.nn.functional as F
from transformers import SegformerForSemanticSegmentation

# Load the trained model 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SegformerForSemanticSegmentation.from_pretrained('nvidia/segformer-b3-finetuned-ade-512-512')

# Replace with the actual number of classes
model.config.num_labels = 2  

# Load the state from the fine-tuned model and set to model.eval() mode
model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
model.to(device)
model.eval()

# Video inference
cap = cv2.VideoCapture('/kaggle/input/video-test-2/videoplayback.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/kaggle/working/output_video.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Perform transformations
data_transforms = TF.Compose([
    TF.ToPILImage(),
    TF.Resize((360, 640)),
    TF.ToTensor(),
    TF.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Inference loop 
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # Preprocess the frame
        input_tensor = data_transforms(frame).unsqueeze(0).to(device)
        
        with torch.no_grad():
            outputs = model(pixel_values=input_tensor,return_dict=True)
            outputs = F.interpolate(outputs["logits"], size=(360, 640), mode="bilinear", align_corners=False)
            
            preds = torch.argmax(outputs, dim=1)
            preds = torch.unsqueeze(preds, dim=1)
            predicted_mask = (torch.sigmoid(preds) > 0.5).float()

        # Create an RGB version of the mask to overlay on the original frame
        mask_np = predicted_mask.cpu().squeeze().numpy()
        mask_resized = cv2.resize(mask_np, (frame.shape[1], frame.shape[0]))
        
        # Modify this section to create a green mask
        mask_rgb = np.zeros((mask_resized.shape[0], mask_resized.shape[1], 3), dtype=np.uint8)
        mask_rgb[:, :, 1] = (mask_resized * 255).astype(np.uint8)  # Set only the green channel

        # Post-processing for mask smoothening
        # Remove noise
        kernel = np.ones((3,3), np.uint8)
        opening = cv2.morphologyEx(mask_rgb, cv2.MORPH_OPEN, kernel, iterations=2)
        
        # Close small holes
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)

        # Overlay the mask on the frame
        blended = cv2.addWeighted(frame, 0.65, closing, 0.6, 0)
        
        # Write the blended frame to the output video
        out.write(blended)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)

In [ ]:
len(np.unique(image))